# 1. Ablation testing using pretrained huggingface t5-small


Source github: https://github.com/Shivanandroy/T5-Finetuning-PyTorch

In [1]:
# install libraries
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

You should consider upgrading via the 'c:\users\cedri\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\cedri\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\cedri\miniconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\cedri\miniconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModel

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

#### Check data

In [3]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv")

In [4]:
df.sample(10)

,headlines,text
29574,India ought to be one of our closest partners:...,"India ""ought to be"" one of US' closest partner..."
68868,PM Modi releases postage stamp on Lord Ram in ...,Prime Minister Narendra Modi has released a po...
76670,Mumbai to get new cruise terminal featuring ho...,The Mumbai Port Trust (MbPT) has awarded contr...
25305,YouTube now allows users to search videos usin...,Google-owned video sharing platform YouTube no...
70388,Taapsee Pannu responds to user trolling her bi...,Taapsee Pannu responded to a Twitter user who ...
27540,ICC to give strict ball tampering bans post Au...,ICC has decided to hand stricter bans for ball...
25404,"Dhoni not the same person he was, age catching...",Following India's series defeat against Englan...
94975,New York's Eleven Madison Park named world's b...,Eleven Madison Park in New York has topped the...
51707,Woman player awarded 'Spirit of Cricket' for c...,England's woman cricketer Anya Shrubsole won t...
59420,Sunny Leone says Jet Airways delays ruined 'we...,Actress Sunny Leone and her husband Daniel Web...


#### Display 

In [5]:
# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

#### Custom dataset class

In [6]:
class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

#### Train and validate code

In [7]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

def validate(epoch, tokenizer, model, device, loader):
    """
    Function to evaluate model for predictions
    """
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            
            
            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
            
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%10==0:
                
                console.print(f'Completed {_}')
                
                
            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals


#### T5 Trainer

In [8]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

#### Model paramters

In [9]:

# let's define model parameters specific to T5
model_params = {
    "MODEL": "t5-small",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 4,  # training batch size
    "VALID_BATCH_SIZE": 4,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [10]:
# let's get a news summary dataset
# dataframe has 2 columns: 
#   - text: long article content
#   - headlines: one line summary of news
path = "https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv"

df = pd.read_csv(path)


# T5 accepts prefix of the task to be performed:
# Since we are summarizing, let's add summarize to source text as a prefix
df["text"] = "summarize: " + df["text"]

In [11]:
T5Trainer(dataframe=df[:500], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

[12:41:01] [Model]: Loading t5-small...           ]8;id=760384;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\689430518.py\CreatorTemp/ipykernel_34632/689430518.py]8;;\:]8;id=770144;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\689430518.py#16\16]8;;\
                                                                                             

[12:41:14] [Data]: Reading data...                ]8;id=403549;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\689430518.py\CreatorTemp/ipykernel_34632/689430518.py]8;;\:]8;id=415563;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\689430518.py#27\27]8;;\
                                                                                             

                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                source_text                  |                 target_text                 |
|---------------------------------------------+---------------------------------------------|
|summarize: Saurav Kant, an alumnus of upGrad | upGrad learner switches to career in ML & Al|
|and IIIT-B's PG Program in Machine learning  |             with 90% salary hike            |
|   and Artificial Intelligence, was a Sr     |                                             |
| Systems Engineer at Infosys with almost 5   |                                             |
| years of work experience. The program and   |                                             |
| upGrad's 360-degree career support helped   |                                             |
| him transition to a Data Scientist at Tech  |                                             |
|  Mahindra with 90% salary hike. upGrad's    |                                             |
| Online Power Learning has powered 3 lakh+   |                                             |
|                  careers.                   |                                             |
|  summarize: Kunal Shah's credit card bill   | Delhi techie wins free food from Swiggy for |
|payment platform, CRED, gave users a chance  |               one year on CRED              |
| to win free food from Swiggy for one year.  |                                             |
|Pranav Kaushik, a Delhi techie, bagged this  |                                             |
|reward after spending 2000 CRED coins. Users |                                             |
| get one CRED coin per rupee of bill paid,   |                                             |
|  which can be used to avail rewards from    |                                             |
|  brands like Ixigo, BookMyShow, UberEats,   |                                             |
|             Cult.Fit and more.              |                                             |
+-------------------------------------------------------------------------------------------+

FULL Dataset: (500, 2)

TRAIN Dataset: (400, 2)

...

Completed 20

[12:42:35] [Validation Completed.]               ]8;id=103765;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\689430518.py\CreatorTemp/ipykernel_34632/689430518.py]8;;\:]8;id=6068;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\689430518.py#105\105]8;;\
                                                                                             

[Model] Model saved @ outputs\model_files

[Validation] Generation on Validation data saved @ outputs\predictions.csv

[Logs] Logs saved @ outputs\logs.txt

In [12]:
print("Done")

Done


### Check validation set

In [13]:
import pandas as pd
df_val = pd.read_csv("outputs/predictions.csv")

df_val[:10]

,Unnamed: 0,Generated Text,Actual Text
0,0,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
1,1,Sunrise Lee gave doctor lap dance to persuade ...,Pharma exec gave doctor a lap dance to sell me...
2,2,PM Modi says opposition talks only about Modi ...,I think the opposition even dreams about me: P...
3,3,Indian Space Research Organisation unveils hum...,ISRO unveils Bengaluru centre for manned space...
4,4,Rahul Gandhi replies to Goa CM's letter accusi...,CM Parrikar under pressure from PM after our G...
5,5,eUFS will let smartphones store 260 videos sho...,Samsung builds world's first 1TB storage chip ...
6,6,woman got leg stuck inside toilet hole after f...,Railway police rescues woman with leg stuck in...
7,7,"the move is a ""response to the illegal migrati...",US begins sending back asylum seekers to Mexico
8,8,Yashwant Sinha demands probe into loans worth ...,Yashwant Sinha demands probe into alleged fund...
9,9,text,headlines


## Prune

Load a pretrained t5-small model from transformers

In [256]:
from transformers import T5Model

In [257]:
model = T5Model.from_pretrained(model_params["MODEL"])
model = model.to(device)

Check the model architecture

In [250]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [259]:
# Heads to remove as a dict. The key represents the layer number and values is an array of the heads to remove
# Here 0: [0, 1] means remove from layer 0, head 0 and 1
heads = { 0: [0, 1], 1: [0, 1] }

model = model.prune_heads(heads)

model

AttributeError: 'T5Stack' object has no attribute 'layer'

#### New trainer

In [16]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)
    heads = { 1: [0, 1], 2: [0, 2] }

    model.prune_heads = heads

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "ablation_predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [17]:
T5Trainer(dataframe=df[:500], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

[12:42:45] [Model]: Loading t5-small...          ]8;id=35809;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\3436021337.py\CreatorTemp/ipykernel_34632/3436021337.py]8;;\:]8;id=99484;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\3436021337.py#16\16]8;;\
                                                                                             

[12:42:57] [Data]: Reading data...               ]8;id=239486;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\3436021337.py\CreatorTemp/ipykernel_34632/3436021337.py]8;;\:]8;id=819082;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\3436021337.py#30\30]8;;\
                                                                                             

                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                source_text                  |                 target_text                 |
|---------------------------------------------+---------------------------------------------|
|summarize: Saurav Kant, an alumnus of upGrad | upGrad learner switches to career in ML & Al|
|and IIIT-B's PG Program in Machine learning  |             with 90% salary hike            |
|   and Artificial Intelligence, was a Sr     |                                             |
| Systems Engineer at Infosys with almost 5   |                                             |
| years of work experience. The program and   |                                             |
| upGrad's 360-degree career support helped   |                                             |
| him transition to a Data Scientist at Tech  |                                             |
|  Mahindra with 90% salary hike. upGrad's    |                                             |
| Online Power Learning has powered 3 lakh+   |                                             |
|                  careers.                   |                                             |
|  summarize: Kunal Shah's credit card bill   | Delhi techie wins free food from Swiggy for |
|payment platform, CRED, gave users a chance  |               one year on CRED              |
| to win free food from Swiggy for one year.  |                                             |
|Pranav Kaushik, a Delhi techie, bagged this  |                                             |
|reward after spending 2000 CRED coins. Users |                                             |
| get one CRED coin per rupee of bill paid,   |                                             |
|  which can be used to avail rewards from    |                                             |
|  brands like Ixigo, BookMyShow, UberEats,   |                                             |
|             Cult.Fit and more.              |                                             |
+-------------------------------------------------------------------------------------------+

FULL Dataset: (500, 2)

TRAIN Dataset: (400, 2)

...

Completed 20

[12:44:31] [Validation Completed.]              ]8;id=167510;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\3436021337.py\CreatorTemp/ipykernel_34632/3436021337.py]8;;\:]8;id=239009;file://C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_34632\3436021337.py#108\108]8;;\
                                                                                             

[Model] Model saved @ outputs\model_files

[Validation] Generation on Validation data saved @ outputs\predictions.csv

[Logs] Logs saved @ outputs\logs.txt

#### Calculate BLEU score

In [206]:
from torchtext.data.utils import get_tokenizer

# Place-holders
token_transform = {}
vocab_transform = {}

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TRG_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

In [207]:
# tokenize for example
#example of tokenization of the english part
print("English sentence: ", df['text'][0])
print("Tokenization: ", token_transform[TRG_LANGUAGE](df['text'][0]))

English sentence:  Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.
Tokenization:  ['Saurav', 'Kant', ',', 'an', 'alumnus', 'of', 'upGrad', 'and', 'IIIT', '-', 'B', "'s", 'PG', 'Program', 'in', 'Machine', 'learning', 'and', 'Artificial', 'Intelligence', ',', 'was', 'a', 'Sr', 'Systems', 'Engineer', 'at', 'Infosys', 'with', 'almost', '5', 'years', 'of', 'work', 'experience', '.', 'The', 'program', 'and', 'upGrad', "'s", '360', '-', 'degree', 'career', 'support', 'helped', 'him', 'transition', 'to', 'a', 'Data', 'Scientist', 'at', 'Tech', 'Mahindra', 'with', '90', '%', 'salary', 'hike', '.', 'upGrad', "'s", 'Online', 'Power', 'Learning', 'has', 'powered', '3', 'lakh+

In [237]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(iterator, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for src, trg in iterator:
        
#         pred_trg, _ = translate_sentence(src, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = token_transform[SRC_LANGUAGE](src.rstrip("\n"))
        
        #tokenize target sentence so it can be compared with pred_trgs
        trg = token_transform[TRG_LANGUAGE](trg.rstrip("\n"))
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
#         print(pred_trgs)
    
    return bleu_score(pred_trgs, trgs)

Test with 1 sentence

In [209]:
test_head = df['headlines'][0]
test_text = df['text'][0]

test_h_token = token_transform[SRC_LANGUAGE](test_head.rstrip("\n"))
test_t_token = token_transform[TRG_LANGUAGE](test_text.rstrip("\n"))

print(test_h_token)
print('---')
print(test_t_token)

['upGrad', 'learner', 'switches', 'to', 'career', 'in', 'ML', '&', 'Al', 'with', '90', '%', 'salary', 'hike']
---
['Saurav', 'Kant', ',', 'an', 'alumnus', 'of', 'upGrad', 'and', 'IIIT', '-', 'B', "'s", 'PG', 'Program', 'in', 'Machine', 'learning', 'and', 'Artificial', 'Intelligence', ',', 'was', 'a', 'Sr', 'Systems', 'Engineer', 'at', 'Infosys', 'with', 'almost', '5', 'years', 'of', 'work', 'experience', '.', 'The', 'program', 'and', 'upGrad', "'s", '360', '-', 'degree', 'career', 'support', 'helped', 'him', 'transition', 'to', 'a', 'Data', 'Scientist', 'at', 'Tech', 'Mahindra', 'with', '90', '%', 'salary', 'hike', '.', 'upGrad', "'s", 'Online', 'Power', 'Learning', 'has', 'powered', '3', 'lakh+', 'careers', '.']


In [210]:
test_df

,Unnamed: 0,Generated Text,Actual Text
0,0,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
1,1,Sunrise Lee gave doctor lap dance to persuade ...,Pharma exec gave doctor a lap dance to sell me...
2,2,PM Modi says opposition talks only about Modi ...,I think the opposition even dreams about me: P...
3,3,Indian Space Research Organisation unveils hum...,ISRO unveils Bengaluru centre for manned space...
4,4,Rahul Gandhi replies to Goa CM's letter accusi...,CM Parrikar under pressure from PM after our G...
...,...,...,...
95,95,BMTC bus conductor returned bag of passenger w...,B'luru bus conductor returns bag containing â1...
96,96,America's David Matheson divorced wife of 34 y...,US man who claimed to 'cure' gays with therapy...
97,97,Pierre-Hugues Herbert and Nicolas Mahut beat F...,"Frenchmen Herbert, Mahut win Aus Open men's do..."
98,98,Subramanian Swamy claims Priyanka suffers from...,"Priyanka Gandhi has bipolar disorder, beats up..."


In [211]:
test_df = pd.read_csv('outputs/predictions.csv')

test_iter = []

for a in test_df.iterrows():
    test_iter.append((a[1]['Generated Text'], a[1]['Actual Text']))
    
    
test_iter[0]

("Kunal Shah's credit card bill payment platform gives users a chance to win free food from Swiggy",
 'Delhi techie wins free food from Swiggy for one year on CRED')

In [212]:
# test_iter = Multi30k(split=('test'), language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
bleu_score = calculate_bleu(test_iter, model, device)

print(f'BLEU score | base model = {bleu_score*100:.2f}')

BLEU score | base model = 10.17


In [239]:
test_df = pd.read_csv('outputs/ablation_predictions.csv')
test_df = test_df.drop([9])

test_iter = []

for a in test_df.iterrows():
    test_iter.append((a[1]['Generated Text'], a[1]['Actual Text']))
    
    
# test_df[:60]

In [238]:
# test_iter = Multi30k(split=('test'), language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
bleu_score = calculate_bleu(test_iter, model, device)

print(f'BLEU score | ablation model = {bleu_score*100:.2f}')

BLEU score | ablation model = 10.17
